<a href="https://colab.research.google.com/github/FaizanMostafa/ai/blob/main/IMDB_Movie_Review_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

100%|██████████| 25.7M/25.7M [00:00<00:00, 129MB/s]

Extracting files...


In [ ]:
# prompt: load data from path into data frame

import pandas as pd

try:
  df = pd.read_csv(path + '/IMDB Dataset.csv')
  print(df.head())
except FileNotFoundError:
  print(f"Error: File not found at path: {path}/IMDB Dataset.csv")
except pd.errors.ParserError:
  print(f"Error: Could not parse the file at path: {path}/IMDB Dataset.csv. Check if it's a valid CSV file.")
except Exception as e:
  print(f"An unexpected error occurred: {e}")


                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [ ]:
# prompt: remove html tags from review column

import re

def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

df['review'] = df['review'].apply(remove_html_tags)
print(df.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. The filming tec...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [ ]:
df = df.iloc[:10000]
df.shape

(10000, 2)

In [ ]:
df["sentiment"].value_counts()


,count
sentiment,
positive,5028
negative,4972


In [ ]:
df.isnull().sum()

,0
review,0
sentiment,0


In [ ]:
print(df.duplicated().sum())
df.drop_duplicates(inplace=True)
print(df.duplicated().sum())

17
0


In [ ]:
# Convert 'review' column to lowercase
df['review'] = df['review'].str.lower()

In [ ]:
# prompt: remove stop words from dataframe review column

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

# Function to remove stopwords from a string
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return " ".join(filtered_words)

print(stop_words)
# Apply the function to the 'review' column
df['review'] = df['review'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'when', 'down', 'against', 'yourself', 'do', "she's", 'about', 'this', 'is', 'under', 'hasn', 'him', 'what', 'of', 'where', 'both', 'ma', "wouldn't", 'mightn', 'if', 't', 're', 'it', 'that', 'most', 'd', 'but', 'y', 'between', 'above', 'having', 'those', 'in', 'doesn', 'has', 'just', 'how', 'into', "you'll", 'his', 'themselves', 'hadn', "couldn't", 'during', 'am', 'few', 'which', 's', 'ourselves', "you'd", 'only', 'couldn', 'other', 'll', 'your', 'there', 'was', 'through', 'me', 'will', 'can', 'had', 'yours', 'being', "wasn't", 'while', "mightn't", 'wasn', "weren't", "shouldn't", 'than', 'such', 'at', 'before', 'so', "isn't", "you've", "didn't", 'her', 'off', 'are', 'our', 've', 'theirs', 'more', 'same', 'haven', 'all', 'them', "won't", 'she', 'doing', 'shouldn', 'once', 'for', 'each', 'up', 'very', 'himself', 'didn', 'weren', 'with', 'were', 'after', 'to', "mustn't", 'not', "hasn't", "it's", 'we', 'an', 'further', 'by', "hadn't", 'ain', 'wouldn', 'or', 'have', 'herself', 'these', 'di

In [ ]:
df["review"][0]

"one reviewers mentioned watching 1 oz episode hooked. right, exactly happened me.the first thing struck oz brutality unflinching scenes violence, set right word go. trust me, show faint hearted timid. show pulls punches regards drugs, sex violence. hardcore, classic use word.it called oz nickname given oswald maximum security state penitentary. focuses mainly emerald city, experimental section prison cells glass fronts face inwards, privacy high agenda. em city home many..aryans, muslims, gangstas, latinos, christians, italians, irish more....so scuffles, death stares, dodgy dealings shady agreements never far away.i would say main appeal show due fact goes shows dare. forget pretty pictures painted mainstream audiences, forget charm, forget romance...oz mess around. first episode ever saw struck nasty surreal, say ready it, watched more, developed taste oz, got accustomed high levels graphic violence. violence, injustice (crooked guards who'll sold nickel, inmates who'll kill order g

In [ ]:
# prompt: extract dependent and independent variable from dataframe.

X = df['review']  # Independent variable (features)
y = df['sentiment']  # Dependent variable (target)

In [ ]:
X.head()

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."


In [ ]:
# prompt: encode y

from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
le = LabelEncoder()

# Fit and transform the 'sentiment' column
y = le.fit_transform(y)

y

array([1, 1, 1, ..., 0, 0, 1])

In [ ]:
# prompt: split the data into 20% for training

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# prompt: Assume you have X_train, X_test, y_train, y_test. Now apply bag of word on the data

from sklearn.feature_extraction.text import CountVectorizer

# Create a CountVectorizer object
vectorizer = CountVectorizer()

# Fit and transform the training data
X_train_bow = vectorizer.fit_transform(X_train)

# Transform the testing data
X_test_bow = vectorizer.transform(X_test)

In [ ]:
X_train_bow.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
X_test_bow.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
from sklearn.naive_bayes import GaussianNB

# Initialize a Multinomial Naive Bayes classifier
gnb_model = GaussianNB()

# Train the model
gnb_model.fit(X_train_bow.toarray(), y_train)

# Make predictions on the test set
y_pred = gnb_model.predict(X_test_bow.toarray())

from sklearn.metrics import accuracy_score, classification_report

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print(classification_report(y_test, y_pred))

Accuracy: 0.627941912869304
              precision    recall  f1-score   support

           0       0.61      0.70      0.65       985
           1       0.66      0.55      0.60      1012

    accuracy                           0.63      1997
   macro avg       0.63      0.63      0.63      1997
weighted avg       0.63      0.63      0.63      1997



In [ ]:
# prompt: TRAIN MACHINE LERANING MODEL WITH DATA, ASSUME YOU HAVE.

from sklearn.naive_bayes import MultinomialNB

# Initialize a Multinomial Naive Bayes classifier
mnb_model = MultinomialNB()

# Train the model
mnb_model.fit(X_train_bow, y_train)

# Make predictions on the test set
y_pred = mnb_model.predict(X_test_bow)

from sklearn.metrics import accuracy_score, classification_report

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print(classification_report(y_test, y_pred))

Accuracy: 0.8587881822734101
              precision    recall  f1-score   support

           0       0.84      0.88      0.86       985
           1       0.88      0.84      0.86      1012

    accuracy                           0.86      1997
   macro avg       0.86      0.86      0.86      1997
weighted avg       0.86      0.86      0.86      1997



In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize a Multinomial Naive Bayes classifier
rfc_model = RandomForestClassifier()

# Train the model
rfc_model.fit(X_train_bow, y_train)

# Make predictions on the test set
y_pred = rfc_model.predict(X_test_bow)

from sklearn.metrics import accuracy_score, classification_report

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print(classification_report(y_test, y_pred))

Accuracy: 0.8537806710065098
              precision    recall  f1-score   support

           0       0.85      0.85      0.85       985
           1       0.86      0.85      0.86      1012

    accuracy                           0.85      1997
   macro avg       0.85      0.85      0.85      1997
weighted avg       0.85      0.85      0.85      1997



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Create a CountVectorizer object
vectorizer = CountVectorizer(max_features=3000)

# Fit and transform the training data
X_train_bow = vectorizer.fit_transform(X_train).toarray()

# Transform the testing data
X_test_bow = vectorizer.transform(X_test).toarray()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize a Multinomial Naive Bayes classifier
rfc_model = RandomForestClassifier()

# Train the model
rfc_model.fit(X_train_bow, y_train)

# Make predictions on the test set
y_pred = rfc_model.predict(X_test_bow)

from sklearn.metrics import accuracy_score, classification_report

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print(classification_report(y_test, y_pred))

Accuracy: 0.8362543815723585
              precision    recall  f1-score   support

           0       0.82      0.85      0.84       985
           1       0.85      0.82      0.84      1012

    accuracy                           0.84      1997
   macro avg       0.84      0.84      0.84      1997
weighted avg       0.84      0.84      0.84      1997



In [ ]:
# Create a CountVectorizer object
vectorizer = CountVectorizer(ngram_range=(2,2),max_features=5000)

# Fit and transform the training data
X_train_bow = vectorizer.fit_transform(X_train).toarray()

# Transform the testing data
X_test_bow = vectorizer.transform(X_test).toarray()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize a Multinomial Naive Bayes classifier
rfc_model = RandomForestClassifier()

# Train the model
rfc_model.fit(X_train_bow, y_train)

# Make predictions on the test set
y_pred = rfc_model.predict(X_test_bow)

from sklearn.metrics import accuracy_score, classification_report

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print(classification_report(y_test, y_pred))

Accuracy: 0.7496244366549825
              precision    recall  f1-score   support

           0       0.72      0.80      0.76       985
           1       0.79      0.70      0.74      1012

    accuracy                           0.75      1997
   macro avg       0.75      0.75      0.75      1997
weighted avg       0.75      0.75      0.75      1997



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TfidfVectorizer object
tfidf_vectorizer = TfidfVectorizer(max_features=3000)

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train).toarray()

# Transform the testing data
X_test_tfidf = tfidf_vectorizer.transform(X_test).toarray()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize a Multinomial Naive Bayes classifier
rfc_model = RandomForestClassifier()

# Train the model
rfc_model.fit(X_train_bow, y_train)

# Make predictions on the test set
y_pred = rfc_model.predict(X_test_bow)

from sklearn.metrics import accuracy_score, classification_report

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print(classification_report(y_test, y_pred))

Accuracy: 0.7371056584877316
              precision    recall  f1-score   support

           0       0.71      0.79      0.75       985
           1       0.77      0.68      0.72      1012

    accuracy                           0.74      1997
   macro avg       0.74      0.74      0.74      1997
weighted avg       0.74      0.74      0.74      1997

